<a href="https://colab.research.google.com/github/nate-walter/colab-starcoder/blob/main/StarCoder_W_ToT_DataProcsng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/starCoder_project

/content/drive/MyDrive/starCoder_project


In [ ]:
%cd starcoder

/content/drive/MyDrive/starCoder_project/starcoder


In [ ]:
## Uncomment if for some reason ToT repo not still in path /content/drive/MyDrive/starCoder_project/starcoder

# !git clone https://github.com/princeton-nlp/tree-of-thought-llm.git

In [ ]:
!pip install --quiet -r requirements.txt   # This allows you to connect with HuggingFace and sign in
!pip install --quiet bitsandbytes
!pip install --quiet git+https://github.com/huggingface/transformers.git
#!pip install --quiet --upgrade transformers # trying to use the github link to see if it works better (2023-09-11 11:45)
!pip install --quiet --upgrade accelerate
!pip install --quiet sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pypr

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import ast
import os
import textwrap

class FunctionAndClassExtractor:

    def __init__(self):
        self.functions = []
        self.classes = []
        self.repo_name = "tree-of-thought-llm"  # Hardcoded for now
        self.stars_count = 3000  # Hardcoded for now

    def extract_from_file(self, filepath):
        with open(filepath, 'r') as f:
            source = f.read()

        tree = ast.parse(source)

        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                function = {
                    'name': node.name,
                    'params': [a.arg for a in node.args.args],
                    'body': textwrap.indent('\n'.join(source.splitlines()[node.lineno-1:node.end_lineno]), '  '),
                    'docstring': ast.get_docstring(node),
                    'file_path': filepath,
                    'repo_name': self.repo_name,
                    'stars_count': self.stars_count
                }
                self.functions.append(function)

            elif isinstance(node, ast.ClassDef):
                class_name = node.name
                methods = []
                for class_node in ast.iter_child_nodes(node):
                    if isinstance(class_node, ast.FunctionDef):
                        method = {
                            'name': class_node.name,
                            'params': [a.arg for a in class_node.args.args],
                            'body': textwrap.indent('\n'.join(source.splitlines()[node.lineno-1:node.end_lineno]), '  '),
                            'docstring': ast.get_docstring(class_node),
                            'file_path': filepath,
                            'repo_name': self.repo_name,
                            'stars_count': self.stars_count
                        }
                        methods.append(method)

                class_info = {
                    'name': class_name,
                    'methods': methods,
                    'docstring': ast.get_docstring(node),
                    'repo_name': self.repo_name,
                    'stars_count': self.stars_count
                }

                self.classes.append(class_info)


    def process_dir(self, dirpath):
        for root, dirs, files in os.walk(dirpath):
            for filename in files:
                if filename.endswith('.py'):
                    self.extract_from_file(os.path.join(root, filename))

    def display_collected_data(self):
      pass
        # print("\nCollected Functions:") ## Uncomment to see the funcs and classes
        # for function in self.functions:
        #     print(function)

        # print("\nCollected Classes:")
        # for class_info in self.classes:
        #     print(class_info)



# Initialize the extractor and process directory
extractor = FunctionAndClassExtractor()
extractor.process_dir('tree-of-thought-llm')
#extractor.display_collected_data() ## Uncomment to see print out of funcs and classes gathered


# Existing imports and FunctionAndClassExtractor class definition

def format_data(functions, classes):
    formatted_data = []

    for function in functions:
        repo_name = function['repo_name']
        stars_count = function['stars_count']
        file_path = function['file_path']
        code_content = function['body']

        record = f'<reponame>{repo_name}<gh_stars>{stars_count}<filename>{file_path} {code_content}'
        formatted_data.append(record)

    for class_info in classes:
        repo_name = class_info['repo_name']
        stars_count = class_info['stars_count']
        class_name = class_info['name']

        record = f'<reponame>{repo_name}<gh_stars>{stars_count}<classname>{class_name}'
        formatted_data.append(record)

    return formatted_data


# Format the extracted data
formatted_data = format_data(extractor.functions, extractor.classes)

# For demonstration, let's print the first few formatted records
print("First formatted records:")
for record in formatted_data[:1]:
    print(record)


First formatted records:
<reponame>tree-of-thought-llm<gh_stars>3000<filename>tree-of-thought-llm/run.py   def run(args):
      task = get_task(args.task)
      logs, cnt_avg, cnt_any = [], 0, 0
      if args.naive_run:
          file = f'./logs/{args.task}/{args.backend}_{args.temperature}_naive_{args.prompt_sample}_sample_{args.n_generate_sample}_start{args.task_start_index}_end{args.task_end_index}.json'
      else:
          file = f'./logs/{args.task}/{args.backend}_{args.temperature}_{args.method_generate}{args.n_generate_sample}_{args.method_evaluate}{args.n_evaluate_sample}_{args.method_select}{args.n_select_sample}_start{args.task_start_index}_end{args.task_end_index}.json'
      os.makedirs(os.path.dirname(file), exist_ok=True)

      for i in range(args.task_start_index, args.task_end_index):
          # solve
          if args.naive_run:
              ys, info = naive_solve(args, task, i) 
          else:
              ys, info = solve(args, task, i)

          # log
      

In [ ]:
#!pip install transformers datasets

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Initialize the quantized model
name = "bigcode/starcoderbase-7b"
model_8bit = AutoModelForCausalLM.from_pretrained(name, device_map="auto", load_in_8bit=True)

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(name)

# Set the padding token if it's not already set
if not tokenizer.pad_token:
    tokenizer.pad_token = tokenizer.eos_token

# Freeze all layers
for param in model_8bit.parameters():
    param.requires_grad = False

# Unfreeze the last layer (assuming it's the last layer of the decoder, adjust as needed)
for param in model_8bit.transformer.h[-1].parameters():
    param.requires_grad = True

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using pad_token, but it is not set yet.


RuntimeError: ignored